Model definition

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input, regularizers

def conv_net(x):
    y = layers.Conv2D(16, (5, 5), activation='relu', padding='SAME')(x)
    y = layers.Dropout(0.5)(y)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.Conv2D(32, (5, 5), activation='relu', padding='SAME')(y)
    y = layers.Dropout(0.5)(y)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.Conv2D(64, (5, 5), activation='relu', padding='SAME')(y)
    y = layers.Dropout(0.5)(y)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.Conv2D(128, (5, 5), activation='relu', padding='SAME')(y)
    y = layers.Dropout(0.5)(y)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.Conv2D(256, (5, 5), activation='relu', padding='SAME')(y)
    y = layers.Dropout(0.5)(y)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.Flatten()(y)
    return y

def classifier_net(x):
    y = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    y = layers.Dropout(0.5)(y)
    y = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(y)
    y = layers.Dropout(0.5)(y)
    y = layers.Dense(10, activation='softmax')(y)
    return y

def model():
    inputs = Input(shape=(32,32,1))
    
    conv_output = conv_net(inputs)
    branches_output = [classifier_net(conv_output) for _ in range(2)] 
    merged_output = tf.stack(branches_output, axis=1)
    
    # 创建多输出模型
    model = models.Model(inputs=inputs, outputs=merged_output)
    
    return model


2024-08-11 18:32:41.500854: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-11 18:32:41.502613: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-11 18:32:41.540569: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-11 18:32:41.541222: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-11 18:32:42.293291: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

batch data generator

In [3]:
import h5py
import numpy as np
import random

def get_steps_count(h5_dataset_path, batch_size):
    with h5py.File(h5_dataset_path, 'r') as f:
        num_samples = f['images'].shape[0]
        num_batches = int(np.ceil(num_samples / batch_size))
    return num_batches

def data_generator(h5_dataset_path, batch_size):
    with h5py.File(h5_dataset_path, 'r') as f:
        num_samples = f['images'].shape[0]
        num_batches = int(np.ceil(num_samples / batch_size))
        
        while True:  # 生成器需要无限循环
            index_list = list(range(num_batches))
            random.shuffle(index_list)
            for i in index_list:
                start_index = i * batch_size
                end_index = min((i + 1) * batch_size, num_samples)

                batch_images = f['images'][start_index:end_index]
                batch_labels = f['labels'][start_index:end_index]
                
                yield batch_images, batch_labels

Train

In [4]:
epochs = 60
batch_size = 32

train_data_generator = data_generator('svhn_dataset/svhn_2digits_train.h5',batch_size)
train_steps = get_steps_count('svhn_dataset/svhn_2digits_train.h5',batch_size)
test_data_generator = data_generator('svhn_dataset/svhn_2digits_test.h5',batch_size)
test_steps = get_steps_count('svhn_dataset/svhn_2digits_test.h5',batch_size)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.7, 
    patience=3, 
    min_lr=1e-6, 
    verbose=1)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003, clipvalue=1.0)

model = model()
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_data_generator,
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=test_data_generator,
    validation_steps=test_steps,
    callbacks=[reduce_lr]
)

model.save('models/svhn_2digits_model.keras')

Epoch 1/60
567/567 [==============================] - 14s 22ms/step - loss: 2.7878 - accuracy: 0.1542 - val_loss: 2.4816 - val_accuracy: 0.1839 - lr: 3.0000e-04
Epoch 2/60
567/567 [==============================] - 13s 23ms/step - loss: 2.3250 - accuracy: 0.1819 - val_loss: 2.2684 - val_accuracy: 0.2168 - lr: 3.0000e-04
Epoch 3/60
567/567 [==============================] - 13s 23ms/step - loss: 2.0351 - accuracy: 0.2726 - val_loss: 2.1424 - val_accuracy: 0.2516 - lr: 3.0000e-04
Epoch 4/60
567/567 [==============================] - 13s 23ms/step - loss: 1.8505 - accuracy: 0.3179 - val_loss: 2.0448 - val_accuracy: 0.2978 - lr: 3.0000e-04
Epoch 5/60
567/567 [==============================] - 13s 23ms/step - loss: 1.7087 - accuracy: 0.3668 - val_loss: 1.9465 - val_accuracy: 0.3333 - lr: 3.0000e-04
Epoch 6/60
567/567 [==============================] - 13s 23ms/step - loss: 1.5806 - accuracy: 0.4175 - val_loss: 1.9030 - val_accuracy: 0.3747 - lr: 3.0000e-04
Epoch 7/60
567/567 [==============